In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn
import statistics
import math
from scipy.stats import poisson,skellam

world_cup_matches = pd.read_csv("C:\\Users\\ianfr\\Documents\\dev\\inteligencia-computacional\\dataset\\results.csv")
world_cup_matches['year'] = world_cup_matches['date'].str[:4].astype(int)
world_cup_matches['month'] = world_cup_matches['date'].str[5:7].astype(int)
world_cup_matches['day'] = world_cup_matches['date'].str[8:].astype(int)
world_cup_matches = world_cup_matches[['home_team', 'home_score', 'away_score', 'away_team', 'tournament', 'year', 'month', 'day']]
world_cup_matches = world_cup_matches[(world_cup_matches.year) >= 2015]
world_cup_matches = world_cup_matches[(world_cup_matches.year) <= 2018]

wc_teams = ['Russia', 'Brazil', 'Australia', 'Iran', 'Japan', 'Saudi Arabia', 'South Korea', 'Egypt', 'Morocco', 'Nigeria', 'Senegal', 'Tunisia', 'Costa Rica', 'Mexico', 'Panama', 'Argentina', 'Colombia', 'Peru', 'Uruguay', 'Belgium', 'Croatia', 'Denmark', 'England', 'France', 'Germany', 'Iceland', 'Poland', 'Portugal', 'Serbia', 'Spain', 'Sweden', 'Switzerland']
wc_teams.sort()
#world_cup_matches = world_cup_matches[world_cup_matches.home_team.isin(teams) | world_cup_matches.away_team.isin(teams)]

world_cup_matches.head()

,home_team,home_score,away_score,away_team,tournament,year,month,day
36609,Bahrain,1,0,Jordan,Friendly,2015,1,4
36610,Iran,1,0,Iraq,Friendly,2015,1,4
36611,South Korea,2,0,Saudi Arabia,Friendly,2015,1,4
36612,South Africa,1,0,Zambia,Friendly,2015,1,4
36613,China PR,4,1,Oman,Friendly,2015,1,5


In [2]:
world_cup_matches.describe()

,home_score,away_score,year,month,day
count,3742.000000,3742.000000,3742.000000,3742.000000,3742.000000
mean,1.573223,1.084714,2016.444949,7.070551,15.049973
std,1.569680,1.285719,1.113711,3.190779,8.879508
min,0.000000,0.000000,2015.000000,1.000000,1.000000
25%,0.000000,0.000000,2015.000000,5.000000,8.000000
50%,1.000000,1.000000,2016.000000,7.000000,13.000000
75%,2.000000,2.000000,2017.000000,10.000000,23.750000
max,15.000000,12.000000,2018.000000,12.000000,31.000000


In [3]:
teams2 = world_cup_matches['home_team'].unique().tolist()
teams3 = world_cup_matches['away_team'].unique().tolist()
tournaments = world_cup_matches['tournament'].unique().tolist()
print(tournaments)
game_weight = {'Friendly': 1, 'FIFA World Cup qualification': 2.5, 'AFC Asian Cup qualification': 2.5,
               'UEFA Euro': 3, 'African Cup of Nations qualification': 2.5, 'Copa América': 3,
               'UNCAF Cup': 3, 'UEFA Euro qualification': 2.5, 'AFC Asian Cup': 3, 'Gold Cup qualification': 2.5,
               'Gold Cup': 3, 'Oceania Nations Cup qualification': 2.5, 'Copa América qualification': 2.5,
               'UEFA Nations League': 3, 'Oceania Nations Cup': 3, 'Confederations Cup': 3,
               'African Nations Championship': 3, 'African Cup of Nations': 3, 'FIFA World Cup': 4}

teams = list(set(teams2).union(set(teams3)))
data = {'Team': teams, 'AttackForce': [0 for i in range(len(teams))], 'DefenseForce': [0 for i in range(len(teams))]}
stats = pd.DataFrame(data)

for time in teams:
    team_goals = 0.0
    conceded_goals = 0.0
    team_matches = 0.0
    time_model = world_cup_matches[(world_cup_matches['home_team'] == time) | (world_cup_matches['away_team'] == time)]
    #print(time_model)
    for index, row in time_model.iterrows():
        match_time = 2018 - row['year']
        time_factor = 1  - 0.56*match_time + 0.1*(match_time*match_time)
        #print(time_factor)
        
        if row['tournament'] in game_weight:
            game_weight_factor = game_weight[row['tournament']]
        else:
            game_weight_factor = 2

        #print(game_weight_factor)
        if row['home_team'] == time:
            team_goals += row['home_score'] * time_factor * game_weight_factor
            conceded_goals += row['away_score'] * time_factor * game_weight_factor
            
            #if time == 'Brazil':
            #    print(conceded_goals, row['away_score'], time_factor, game_weight_factor)

        if row['away_team'] == time:
            team_goals += row['away_score'] * time_factor * game_weight_factor
            conceded_goals += row['home_score'] * time_factor * game_weight_factor
            
            #if time == 'Brazil':
            #    print(conceded_goals, row['home_score'], time_factor, game_weight_factor)
            
        team_matches += 1
        
    
    if conceded_goals != 0 and team_goals != 0:
        stats.loc[stats['Team'] == time, 'AttackForce'] = team_goals/(team_matches)
        stats.loc[stats['Team'] == time, 'DefenseForce'] = (team_matches/(conceded_goals))
    else:
        print(time)
        stats = stats[stats['Team'] != time]
        world_cup_matches = world_cup_matches[world_cup_matches['home_team'] != time]
        world_cup_matches = world_cup_matches[world_cup_matches['away_team'] != time]
    
    if team_matches <= 5:
        stats = stats[stats['Team'] != time]
        world_cup_matches = world_cup_matches[world_cup_matches['home_team'] != time]
        world_cup_matches = world_cup_matches[world_cup_matches['away_team'] != time]
    
stats2 = stats.set_index('Team')

print(stats2)

world_cup_matches['HomeAttack'] = [0.0 for i in range(len(world_cup_matches))]
world_cup_matches['HomeDefence'] = [0.0 for i in range(len(world_cup_matches))]
world_cup_matches['AwayAttack'] = [0.0 for i in range(len(world_cup_matches))]
world_cup_matches['AwayDefence'] = [0.0 for i in range(len(world_cup_matches))]

for index, row in world_cup_matches.iterrows():
    #print(stats2.loc[row['home_team'], 'AttackForce'])
    world_cup_matches.loc[index,'HomeAttack'] = (stats2.loc[row['home_team'], 'AttackForce']) #- statistics.mean(stats2['AttackForce'])) / statistics.stdev(stats2['AttackForce'])
    world_cup_matches.loc[index,'HomeDefence'] = (stats2.loc[row['home_team'], 'DefenseForce']) #- statistics.mean(stats2['DefenseForce'])) / statistics.stdev(stats2['DefenseForce'])
    world_cup_matches.loc[index,'AwayAttack'] = (stats2.loc[row['away_team'], 'AttackForce']) #- statistics.mean(stats2['AttackForce'])) / statistics.stdev(stats2['AttackForce'])
    world_cup_matches.loc[index,'AwayDefence'] = (stats2.loc[row['away_team'], 'DefenseForce']) #- statistics.mean(stats2['DefenseForce'])) / statistics.stdev(stats2['DefenseForce'])
world_cup_matches.head()

['Friendly', 'AFC Asian Cup', 'African Cup of Nations', 'ABCS Tournament', 'FIFA World Cup qualification', 'Gold Cup qualification', 'UEFA Euro qualification', 'Windward Islands Tournament', 'COSAFA Cup', 'Copa América', 'African Cup of Nations qualification', 'African Nations Championship', 'CONIFA European Football Cup', 'Island Games', 'Gold Cup', 'EAFF Championship', 'Oceania Nations Cup qualification', 'CECAFA Cup', 'SAFF Cup', 'Copa América qualification', 'CFU Caribbean Cup qualification', 'Oceania Nations Cup', 'Baltic Cup', 'CONIFA World Football Cup', 'AFC Asian Cup qualification', 'Kirin Cup', "King's Cup", 'UEFA Euro', 'World Unity Cup', 'AFF Championship', 'AFC Challenge Cup', 'UNCAF Cup', 'Confederations Cup', 'CFU Caribbean Cup', 'Intercontinental Cup', 'FIFA World Cup', 'AFF Championship qualification', 'UEFA Nations League']
Bahamas
Saint Martin
British Virgin Islands
Vatican City
Cook Islands
Monaco
Kabylia
Turks and Caicos Islands
Cayman Islands
                     

,home_team,home_score,away_score,away_team,tournament,year,month,day,HomeAttack,HomeDefence,AwayAttack,AwayDefence
36609,Bahrain,1,0,Jordan,Friendly,2015,1,4,1.185106,1.869531,1.088491,1.661442
36610,Iran,1,0,Iraq,Friendly,2015,1,4,1.296471,1.966834,0.893774,1.313832
36611,South Korea,2,0,Saudi Arabia,Friendly,2015,1,4,1.246452,1.248993,1.387400,0.674582
36612,South Africa,1,0,Zambia,Friendly,2015,1,4,1.045849,1.573167,1.245238,1.259244
36613,China PR,4,1,Oman,Friendly,2015,1,5,1.034898,1.177036,1.284375,1.600000


In [4]:
stats2.describe()

,AttackForce,DefenseForce
count,235.000000,235.000000
mean,1.268189,0.863113
std,0.702211,0.338157
min,0.100000,0.143816
25%,0.837857,0.636338
50%,1.128889,0.837438
75%,1.515642,1.057757
max,5.666667,1.966834


In [5]:
for team in wc_teams:
    print(team, "Attack: " + str(stats2.loc[team, 'AttackForce']), "Defense: " + str(stats2.loc[team, 'DefenseForce']))

Argentina Attack: 1.7877083333333328 Defense: 0.7666506947771922
Australia Attack: 1.8177083333333333 Defense: 0.8304498269896194
Belgium Attack: 3.4696078431372546 Defense: 0.7842534214977704
Brazil Attack: 2.0871999999999997 Defense: 1.9215987701767876
Colombia Attack: 1.4644680851063827 Defense: 1.1443876308741177
Costa Rica Attack: 1.2429411764705882 Defense: 0.6847475832438239
Croatia Attack: 2.2262 Defense: 0.5656108597285069
Denmark Attack: 1.5547727272727272 Defense: 1.4336917562724019
Egypt Attack: 1.836 Defense: 0.7832898172323759
England Attack: 2.2662745098039214 Defense: 0.8150871024452614
France Attack: 2.274464285714285 Defense: 0.8911521323997456
Germany Attack: 2.1550943396226407 Defense: 0.7949602519874006
Iceland Attack: 1.2279999999999998 Defense: 0.5543237250554325
Iran Attack: 1.2964705882352936 Defense: 1.9668337832626304
Japan Attack: 1.8896296296296289 Defense: 0.8631713554987214
Mexico Attack: 1.4026086956521735 Defense: 0.7812500000000002
Morocco Attack: 2.21

In [31]:
def get_team_stats(team, return_df=False):
    attack = stats2.loc[team, 'AttackForce']
    defence = stats2.loc[team, 'DefenseForce']

    return [attack, defence]

def simulate_goals(team_pred, game, a_team='A Team', b_team='B Team', verbose=False):
    a_goals = 0
    b_goals = 0
    outcome = 0

    random_event = np.random.random()
    #print(random_event)
    counter = 0


    for index, row in pd.DataFrame(data=game).iterrows():
        for x in range(len(row)):
            counter += row[x]
            
            a_goals = index
            b_goals = x
            
            if counter > random_event:
                break
        if counter > random_event:
            break
    
    if verbose == True:
        #print('Random float: {}'.format(random_event))
        print('{} goals: {}'.format(a_team, str(a_goals)))
        print('{} goals: {}\n'.format(b_team, str(b_goals)))
    
    return a_goals, b_goals

def show_game_summary(game, team_pred, a_team='A Team', b_team='B Team', verbose=False):
    a_team_odds = np.sum(np.tril(game, -1))
    b_team_odds = np.sum(np.triu(game, 1))
    draw_odds = np.sum(np.diag(game))
    
    if verbose == True:
        print('{} chance of Win: {}%'.format(a_team, round(a_team_odds*100, 2)))
        print('{} chance of Win: {}%'.format(b_team, round(b_team_odds*100, 2)))
        print('Odds of a draw: {}%'.format(round(draw_odds*100, 2)))
    
    a_goals, b_goals = simulate_goals(team_pred, game, a_team, b_team, verbose=verbose)
    return a_team_odds, b_team_odds, draw_odds, a_goals, b_goals

# simulacao de uma partida, retornando a matriz que nos interessa
def simulate_match(team_pred, a_team, b_team, max_goals=5, verbose=False):
    game = (np.outer(np.array(team_pred[0]), np.array(team_pred[1])))
    show_game_summary(game, team_pred, a_team, b_team, verbose=verbose)
    return game

In [32]:
team_predz = predicao('Iran', 'Brazil')
game = simulate_match(team_predz, 'Iran', 'Brazil')
#pd.DataFrame(game)
show_game_summary(game, team_predz, 'Iran', 'Brazil', verbose=True)

Iran chance of Win: 23.02%
Brazil chance of Win: 44.21%
Odds of a draw: 32.69%
Iran goals: 0
Brazil goals: 2



(0.23015132908599664, 0.44207163043453673, 0.3268972904210259, 0, 2)

In [33]:
pd.DataFrame(game)

,0,1,2,3,4,5
0,0.176245,0.187031,0.099238,0.035104,0.009313,0.001977
1,0.118909,0.126186,0.066954,0.023684,0.006283,0.001334
2,0.040113,0.042568,0.022587,0.007990,0.002120,0.000450
3,0.009021,0.009573,0.005080,0.001797,0.000477,0.000101
4,0.001522,0.001615,0.000857,0.000303,0.000080,0.000017
5,0.000205,0.000218,0.000116,0.000041,0.000011,0.000002


In [47]:
team_pred = predicao('Spain', 'Brazil')
game = simulate_match(team_pred, 'Spain', 'Brazil')
#pd.DataFrame(game)
show_game_summary(game, team_pred, 'Spain', 'Brazil', verbose=True)

Spain chance of Win: 19.81%
Brazil chance of Win: 56.91%
Odds of a draw: 17.64%
Spain goals: 1
Brazil goals: 2



(0.19807822509694945, 0.5690894036129897, 0.17636283558569796, 1, 2)

In [36]:
pd.DataFrame(game)

,0,1,2,3,4,5
0,0.015953,0.042177,0.055753,0.049133,0.032475,0.017171
1,0.023839,0.063025,0.083313,0.073420,0.048527,0.025659
2,0.017811,0.047090,0.062247,0.054856,0.036257,0.019171
3,0.008872,0.023455,0.031006,0.027324,0.018060,0.009549
4,0.003314,0.008762,0.011583,0.010208,0.006747,0.003567
5,0.000991,0.002619,0.003462,0.003051,0.002016,0.001066


In [17]:
def predicao(a_team, b_team, max_goals=5):
    a_team_stats = get_team_stats(a_team)
    b_team_stats = get_team_stats(b_team)
    
    a_att = a_team_stats[0]
    a_def = a_team_stats[1]
    b_att = b_team_stats[0]
    b_def = b_team_stats[1]
    
    a_avg = a_att / b_def
    b_avg = b_att / a_def
    
    #print(a_avg)
    #print(b_avg)
    
    team_pred = [[poisson_distribution(avg, k) for k in range(0, max_goals+1)] for avg in [a_avg, b_avg]]
    
    return team_pred

def poisson_distribution(avg, k):
    
    result = (math.exp(-avg)*(avg**k)) / math.factorial(k)
    
    return result
    
predicao('Brazil', 'Japan')

[[0.08909430603737378,
  0.21543536445757555,
  0.26046780273196735,
  0.2099422454954127,
  0.12691322876002067,
  0.06137675662669914],
 [0.37405095061354793,
  0.36782796192436246,
  0.18085425174231579,
  0.05928180881473936,
  0.014573888182711587,
  0.0028662852366862514]]

In [25]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
import math

goal_model_data = pd.concat([world_cup_matches[['home_team','away_team', 'HomeAttack', 'HomeDefence', 'AwayAttack', 'AwayDefence', 'home_score']].rename(
            columns={'home_team':'team', 'away_team':'opponent', 'HomeAttack':'att', 'HomeDefence': 'defe', 'AwayAttack': 'away_att', 'AwayDefence': 'away_def', 'home_score':'goals'}),
           world_cup_matches[['away_team','home_team', 'AwayAttack', 'AwayDefence','HomeAttack', 'HomeDefence', 'away_score']].rename(
            columns={'away_team':'team', 'home_team':'opponent', 'AwayAttack':'att', 'AwayDefence': 'defe', 'HomeAttack': 'away_att', 'HomeDefence': 'away_def','away_score':'goals'})])

goal_2 = goal_model_data.reset_index(drop=True)

def get_numbers_from_array(array):
    nums = []
    for n in array:
        try:
            nums.append(n[0])
        except:
            nums.append(n)
    return nums

def model_metrics(dataframe):
    yreal = []
    ypred = []
    team_pred = []
    i = 0
    yreal = list(dataframe['goals'])
    for index, row in dataframe.iterrows():
        team_pred = predicao(row['team'], row['opponent'])
        ypred.append(np.argmax(team_pred[0]))
    ypred = get_numbers_from_array(ypred)
    #print(ypred)
    #for index, row in dataframe.iterrows():
    #    team_pred = team_preds(poisson_model, row['team'], row['opponent'])
        #print(team_pred)
    #    yreal.append(row['goals'])
    #    ypred.append(team_pred[0].index(max(team_pred[0])))
    #print(yreal)
    #print(ypred)
    rms = math.sqrt(mean_squared_error(yreal, ypred))
    r2 = r2_score(yreal, ypred)
    return rms, r2

metrics = model_metrics(goal_2)
print(metrics)

(1.4323923578324371, 0.0012272781488944506)


In [27]:
def normalize_test_model(y):
    if y > (14/3):
        return 5
    elif y > (11/3):
        return 4
    elif y > (8/3):
        return 3
    elif y > (4/3):
        return 2
    elif y < (2/3):
        return 0
    else:
        return 1
    
def compare_accuracy(df):
    count1 = 0

    for index, row in df.iterrows():
        if row['predicted'] == row['goals']:
            count1 += 1
        else:
            continue

    print('Poisson Model: {}% accurate'.format((count1 / len(test_model_df))*100))

#X = goal_2.drop(columns=['goals', 'team', 'opponent'])
#Y = pd.DataFrame(goal_model_data['goals'])
#print(X)
#print(np.asarray(X))
poisson_y_pred = []
for index, row in goal_2.iterrows():
        team_pred = predicao(row['team'], row['opponent'])
        poisson_y_pred.append(np.argmax(team_pred[0]))
poisson_y_pred = get_numbers_from_array(poisson_y_pred)
#print(poisson_y_pred)
ml_data = [poisson_y_pred]

test_model_df = pd.DataFrame(ml_data, 
                             index=['predicted']).transpose()

test_model_df['goals'] = list(goal_2['goals'])
test_model_df['predicted'] = test_model_df['predicted'].apply(normalize_test_model)
print(goal_2)
print(test_model_df)
compare_accuracy(test_model_df)

                      team              opponent       att      defe  \
0                  Bahrain                Jordan  1.185106  1.869531   
1                     Iran                  Iraq  1.296471  1.966834   
2              South Korea          Saudi Arabia  1.246452  1.248993   
3             South Africa                Zambia  1.045849  1.573167   
4                 China PR                  Oman  1.034898  1.177036   
5                 Cameroon              DR Congo  0.981667  1.016088   
6                Australia                Kuwait  1.817708  0.830450   
7                  Senegal                 Gabon  1.678913  1.192946   
8             Burkina Faso              Eswatini  1.135098  0.898678   
9                 Cameroon          South Africa  0.981667  1.016088   
10              Cape Verde                 Congo  0.991600  0.833333   
11             South Korea                  Oman  1.246452  1.248993   
12            Saudi Arabia              China PR  1.387400  0.67

In [28]:
team_pred = predicao('Iran', 'Iraq')
print(team_pred)

[[0.3727731021676517, 0.3678470359482853, 0.18149303298589908, 0.05969822200234986, 0.014727332717444063, 0.0029065432328857083], [0.6348144480906884, 0.28847398791307044, 0.06554453978856357, 0.009928305210865178, 0.0011279113944281406, 0.000102509670012046]]
